**Imports**

In [ ]:
from pathlib import Path
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
# Principal Component Analysis
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline     
sns.set(color_codes=True)

**Global variables**

In [ ]:
csvFile = 'aalborg_2024-05-24_prepared.csv'
csvFilePath = Path.cwd().parent.joinpath('csv').joinpath(csvFile)

**Data Collection**

In [ ]:
# Import data
dfCollected = pd.read_csv(csvFilePath, sep=";", header=0)
dfCollected.round(2)
# type(dfCollected)
# dfCollected.shape

In [ ]:
dfCollected.columns

**Data Inspection**

In [ ]:
# To display the top 5 rows
dfCollected.head()

In [ ]:
# To display last 5 rows
dfCollected.tail()

In [ ]:
dfCollected.info()

In [ ]:
# Check data some more
dfCollected.describe()

**Feature Selection**

In [ ]:
features = ['s_speed_x', 's_speed_y','s_angle','s_track_position']
X = dfCollected.loc[:, features]
X.shape
type(X)
X

**Define Target**

In [ ]:
target = ['a_acceleration', 'a_brake', 'a_steer']
y = dfCollected.loc[:, target]
y.shape

In [ ]:
pcaTransform = PCA(n_components=2)

In [ ]:
pcaComponents = pcaTransform.fit_transform(X)

**Normalize Data**

In [ ]:
X = StandardScaler().fit_transform(X)

**Feature Extraction**

In [ ]:
principalDf = pd.DataFrame(data=pcaComponents, columns=['PCA1', 'PCA2'])
principalDf.head()
extractedDf = pd.concat( [principalDf, dfCollected], axis=1)
extractedDf.shape
extractedDf.head()

**Data Visualization**

In [ ]:
selection = [ "s_plus5_degrees2caraxis", "s_parallel2caraxis", "s_min5_degrees2caraxis", "s_angle" ]
dfSelected = dfCollected.copy().loc[:, selection]
corrMatrix = dfSelected.corr().round(2)
print(matrix)

In [ ]:
sns.heatmap(corrMatrix, annot=True)
plt.show()